<a href="https://colab.research.google.com/github/shivashankarenagandula/ML_Projects/blob/main/MNIST_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print("shiva shankar is a wastefellow")

shiva shankar is a wastefellow


In [3]:
print(5+5-1
      )

9
